In [32]:
import pandas as pd
import urllib
import requests
import json
import sys
import io
import shapely
import base64
import urllib.request
import functools
import geopandas as gp
import fiona
import numpy as np
import shapely
from fiona.crs import from_epsg
from functools import reduce
import itertools

%matplotlib inline

## start here if you want to replicate downloading and transforming the data, otherwise skip to the bottom

In [33]:
myAPI = '90341a28f02794c77f6e3abb153a4952a0bb480b'

In [34]:
variables = ['S2503_C01_001E','S2503_C01_002E','S2503_C01_003E','S2503_C01_004E','S2503_C01_005E','S2503_C01_006E',
             'S2503_C01_007E' , 'S2503_C01_008E' ,'S2503_C01_009E','S2503_C01_010E','S2503_C01_011E',
             'S2503_C01_012E','S2503_C01_013E']

In [35]:
#read in in the variables available. the info you need is in the 1year ACS data
url = "https://api.census.gov/data/2017/acs/acs5/subject/variables.json"
resp = requests.request('GET', url)
aff1y = json.loads(resp.text)

affkeys = np.array(list(aff1y['variables'].keys()))

#extracting variables of B28002 that contain "broadband"
[(k, aff1y['variables'][k]['label'])  for k in affkeys if k.startswith ("S2503_C01_") ]

[('S2503_C01_041E',
  'Estimate!!Occupied housing units!!Occupied housing units!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!$75 000 or more'),
 ('S2503_C01_040E',
  'Estimate!!Occupied housing units!!Occupied housing units!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!$50 000 to $74 999!!30 percent or more'),
 ('S2503_C01_043E',
  'Estimate!!Occupied housing units!!Occupied housing units!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!$75 000 or more!!20 to 29 percent'),
 ('S2503_C01_042E',
  'Estimate!!Occupied housing units!!Occupied housing units!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!$75 000 or more!!Less than 20 percent'),
 ('S2503_C01_045E',
  'Estimate!!Occupied housing units!!Occupied housing units!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Zero or negative income'),
 ('S2503_C01_044E',
  'E

In [36]:
#subject group url:
#https://api.census.gov/data/2017/acs/acs1/subject/groups/S2503.html

key1 = 'S2503_C01_001E,S2503_C01_002E,S2503_C01_003E,S2503_C01_004E,S2503_C01_005E,S2503_C01_006E,S2503_C01_007E'
key2= ',S2503_C01_008E,S2503_C01_009E,S2503_C01_010E,S2503_C01_011E,S2503_C01_012E,S2503_C01_013E'
year = '2017'

counties = [ 5, 47, 61, 81, 85]

url = "https://api.census.gov/data/2017/acs/acs5/subject?get=NAME," \
    + key1 + key2 + "&for=tract:*&in=state:36&in=county:*&key=" + myAPI
resp = requests.request('GET', url).content
income = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')))

income = income[income['county'].isin(counties)]


income.head(1)

,NAME,S2503_C01_001E,S2503_C01_002E,S2503_C01_003E,S2503_C01_004E,S2503_C01_005E,S2503_C01_006E,S2503_C01_007E,S2503_C01_008E,S2503_C01_009E,S2503_C01_010E,S2503_C01_011E,S2503_C01_012E,S2503_C01_013E,state,county,tract,Unnamed: 17
21,"Census Tract 170.09, Richmond County, New York",1588,0,24,67,62,65,42,135,247,87,379,480,104393,36,85,17009,NaN


In [37]:
income.rename(columns={'S2503_C01_001E' : 'total_count', 'S2503_C01_002E':'bracket_1',
                       'S2503_C01_003E':'bracket_2','S2503_C01_004E':'bracket_3',
                       'S2503_C01_005E':'bracket_4','S2503_C01_006E':'bracket_5'
                       ,'S2503_C01_007E':'bracket_6','S2503_C01_008E':'bracket_7',
                       'S2503_C01_009E':'bracket_8','S2503_C01_010E': 'bracket_9', 
                       'S2503_C01_011E':'bracket10','S2503_C01_012E':'bracket_11',
                       'S2503_C01_013E':'median_household_income'}, inplace=True)

In [38]:
income.head(1)

,NAME,total_count,bracket_1,bracket_2,bracket_3,bracket_4,bracket_5,bracket_6,bracket_7,bracket_8,bracket_9,bracket10,bracket_11,median_household_income,state,county,tract,Unnamed: 17
21,"Census Tract 170.09, Richmond County, New York",1588,0,24,67,62,65,42,135,247,87,379,480,104393,36,85,17009,NaN


In [39]:
path = '/Users/katharinevoorhees/CAPSTONE/taxizone_censustract_key.csv'

key = pd.read_csv(path)

key.head()

,index_right,OBJECTID,zone,tractid,tractnum
0,1968,2.0,Jamaica Bay,36081107202,107202
1,222,3.0,Allerton/Pelham Gardens,36005031000,31000
2,224,3.0,Allerton/Pelham Gardens,36005031400,31400
3,226,3.0,Allerton/Pelham Gardens,36005031800,31800
4,225,3.0,Allerton/Pelham Gardens,36005031600,31600


In [40]:
income_taxi = income.merge(key, left_on='tract', right_on='tractnum')

In [41]:
income_taxi.head()

,NAME,total_count,bracket_1,bracket_2,bracket_3,bracket_4,bracket_5,bracket_6,bracket_7,bracket_8,...,median_household_income,state,county,tract,Unnamed: 17,index_right,OBJECTID,zone,tractid,tractnum
0,"Census Tract 170.09, Richmond County, New York",1588,0,24,67,62,65,42,135,247,...,104393,36,85,17009,NaN,2121,5.0,Arden Heights,36085017009,17009
1,"Census Tract 207, Richmond County, New York",1787,379,122,142,85,36,177,148,234,...,31250,36,85,20700,NaN,2136,187.0,Port Richmond,36085020700,20700
2,"Census Tract 207, Richmond County, New York",1787,379,122,142,85,36,177,148,234,...,31250,36,85,20700,NaN,501,189.0,Prospect Heights,36047020700,20700
3,"Census Tract 207, Kings County, New York",2179,82,25,16,48,84,168,170,130,...,113574,36,47,20700,NaN,2136,187.0,Port Richmond,36085020700,20700
4,"Census Tract 207, Kings County, New York",2179,82,25,16,48,84,168,170,130,...,113574,36,47,20700,NaN,501,189.0,Prospect Heights,36047020700,20700


In [42]:
zone_income = income_taxi.groupby('OBJECTID').sum().reset_index()
zone_income_median = income_taxi.groupby('OBJECTID').median().reset_index()

In [43]:
zone_income = zone_income.drop(columns=['median_household_income'])


In [44]:
zone_income = zone_income.iloc[:, 0:13]
zone_income = zone_income.merge(zone_income_median[['OBJECTID','median_household_income']], on='OBJECTID')
zone_income.tail()

,OBJECTID,total_count,bracket_1,bracket_2,bracket_3,bracket_4,bracket_5,bracket_6,bracket_7,bracket_8,bracket_9,bracket10,bracket_11,median_household_income
256,259.0,29056,1080,1193,2128,1394,1267,2321,3964,4969,3814,3908,3018,52521.5
257,260.0,80409,3352,4583,5980,4443,3714,7928,10290,13055,9472,9871,7721,52368.0
258,261.0,3487,71,28,88,32,39,41,152,367,256,903,1510,125640.5
259,262.0,23650,733,721,705,697,600,1002,2034,2911,2570,4182,7495,84375.0
260,263.0,33964,656,603,1042,890,1175,1451,2006,5049,4255,6637,10200,89057.0


In [45]:
zone_income.to_csv('wages_and_median_income.csv')

In [46]:
import os
os.getcwd()

'/Users/katharinevoorhees/CAPSTONE'

In [47]:
#output = zone_income.rename(columns={'OBJECTID':'taxi_zone', 'bracket_1':'2500', 
#                            'bracket_2':'7500', 'bracket_3':'12500',
#                            'bracket_4': '17500','bracket_5':'22500',
#                            'bracket_6': '30000', 'bracket_7':'42500', 
#                            'bracket_8':'62500','bracket_9':'87500', 
#                            'bracket10':'125000', 'bracket_11':'225000'})

In [48]:
#output.to_csv('taxi_zone_wages_raw.csv')

In [49]:
col = [ 'bracket_1', 'bracket_2', 'bracket_3','bracket_4', 'bracket_5', 'bracket_6', 'bracket_7', 'bracket_8',
       'bracket_9', 'bracket10', 'bracket_11']
for i in col: 
    zone_income[i] = zone_income[i]/zone_income['total_count']

In [50]:
zone_income.set_index('OBJECTID', inplace=True)

In [51]:
zone_income.head()

,total_count,bracket_1,bracket_2,bracket_3,bracket_4,bracket_5,bracket_6,bracket_7,bracket_8,bracket_9,bracket10,bracket_11,median_household_income
OBJECTID,,,,,,,,,,,,,
2.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-666666666.0
3.0,18922,0.041803,0.060036,0.108075,0.047035,0.069126,0.098668,0.111880,0.156432,0.097823,0.123454,0.085667,58910.0
4.0,19981,0.070217,0.083930,0.116010,0.065062,0.063911,0.082528,0.103498,0.139483,0.068065,0.119664,0.087633,57788.0
5.0,9048,0.022104,0.017794,0.033488,0.023873,0.024204,0.057140,0.112290,0.172745,0.106432,0.199934,0.229996,81420.5
6.0,25623,0.050462,0.032627,0.046169,0.041018,0.038325,0.068415,0.087890,0.119658,0.118487,0.151192,0.245756,67361.0


In [52]:
lehd = pd.read_csv('lehd_by_taxi_zone.csv')
lehd = lehd[['w_taxi_zone','h_taxi_zone','S000']]
lehd.head()

,w_taxi_zone,h_taxi_zone,S000
0,3.0,3.0,273
1,3.0,4.0,1
2,3.0,5.0,2
3,3.0,6.0,1
4,3.0,7.0,7


In [53]:
new_wage = lehd.merge(zone_income, right_on = 'OBJECTID', left_on='h_taxi_zone', how='left')
new_wage.tail()

,w_taxi_zone,h_taxi_zone,S000,total_count,bracket_1,bracket_2,bracket_3,bracket_4,bracket_5,bracket_6,bracket_7,bracket_8,bracket_9,bracket10,bracket_11,median_household_income
56530,263.0,259.0,32,29056,0.037170,0.041059,0.073238,0.047976,0.043605,0.079880,0.136426,0.171015,0.131264,0.134499,0.103868,52521.5
56531,263.0,260.0,102,80409,0.041687,0.056996,0.074370,0.055255,0.046189,0.098596,0.127971,0.162357,0.117798,0.122760,0.096022,52368.0
56532,263.0,261.0,11,3487,0.020361,0.008030,0.025237,0.009177,0.011184,0.011758,0.043590,0.105248,0.073416,0.258962,0.433037,125640.5
56533,263.0,262.0,138,23650,0.030994,0.030486,0.029810,0.029471,0.025370,0.042368,0.086004,0.123087,0.108668,0.176829,0.316913,84375.0
56534,263.0,263.0,434,33964,0.019315,0.017754,0.030680,0.026204,0.034595,0.042722,0.059063,0.148657,0.125280,0.195413,0.300318,89057.0


In [54]:
col = [ 'bracket_1', 'bracket_2', 'bracket_3','bracket_4', 'bracket_5', 'bracket_6', 'bracket_7', 'bracket_8',
       'bracket_9', 'bracket10', 'bracket_11']
for i in col: 
    new_wage[i] = new_wage['S000'] * new_wage[i]
    
new_wage.sort_values('h_taxi_zone').tail()

,w_taxi_zone,h_taxi_zone,S000,total_count,bracket_1,bracket_2,bracket_3,bracket_4,bracket_5,bracket_6,bracket_7,bracket_8,bracket_9,bracket10,bracket_11,median_household_income
18331,87.0,263.0,470,33964,9.077847,8.344424,14.419385,12.315982,16.259863,20.079202,27.759392,69.868979,58.881463,91.844011,141.149452,89057.0
49433,232.0,263.0,22,33964,0.424921,0.390590,0.674950,0.576493,0.761100,0.939878,1.299376,3.270463,2.756154,4.299081,6.606996,89057.0
18080,86.0,263.0,2,33964,0.038629,0.035508,0.061359,0.052408,0.069191,0.085443,0.118125,0.297315,0.250559,0.390826,0.600636,89057.0
19311,91.0,263.0,8,33964,0.154517,0.142033,0.245436,0.209634,0.276764,0.341774,0.472500,1.189259,1.002238,1.563302,2.402544,89057.0
56534,263.0,263.0,434,33964,8.382523,7.705276,13.314922,11.372630,15.014427,18.541220,25.633141,64.517312,54.371393,84.809151,130.338005,89057.0


In [55]:
new_wage[new_wage['h_taxi_zone'] == 263]['S000'].sum()

20569

In [56]:
output = new_wage.rename(columns={'OBJECTID':'taxi_zone', 'bracket_1':'2500', 
                            'bracket_2':'7500', 'bracket_3':'12500',
                            'bracket_4': '17500','bracket_5':'22500',
                            'bracket_6': '30000', 'bracket_7':'42500', 
                            'bracket_8':'62500','bracket_9':'87500', 
                            'bracket10':'125000', 'bracket_11':'225000'})

In [57]:
output = output.rename(columns={'S000':'LEHD_total_pair_count','h_taxi_zone':'PUlocationID',
                              'w_taxi_zone':'DOlocationID' }).drop(columns=['total_count'])
output.head()

,DOlocationID,PUlocationID,LEHD_total_pair_count,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000,median_household_income
0,3.0,3.0,273,11.412271,16.389811,29.504545,12.840609,18.871367,26.936423,30.543336,42.705845,26.705581,33.702991,23.387221,58910.0
1,3.0,4.0,1,0.070217,0.083930,0.116010,0.065062,0.063911,0.082528,0.103498,0.139483,0.068065,0.119664,0.087633,57788.0
2,3.0,5.0,2,0.044209,0.035588,0.066976,0.047745,0.048408,0.114279,0.224580,0.345491,0.212865,0.399867,0.459991,81420.5
3,3.0,6.0,1,0.050462,0.032627,0.046169,0.041018,0.038325,0.068415,0.087890,0.119658,0.118487,0.151192,0.245756,67361.0
4,3.0,7.0,7,0.257919,0.285195,0.333903,0.297581,0.234586,0.423943,0.596832,0.848628,0.724122,1.033718,1.963572,79656.0


In [62]:
os.getcwd()

'/Users/katharinevoorhees/CAPSTONE'

In [65]:
os.chdir('/Users/katharinevoorhees/Downloads')
output.to_csv('taxi_zone_wages_raw.csv', index=False)

In [66]:
output.head(?)

SyntaxError: invalid syntax (<ipython-input-66-d739bb454e78>, line 1)

## start here if you already have table csv

In [59]:
zone_income.to_csv('zone_income_probabilities.csv')

In [60]:
df = pd.read_csv('zone_income_probabilities.csv')
df.set_index('OBJECTID', inplace=True)

In [61]:
#define wage brackets

bucket = [1,2,3,4,5,6,7,8,9,10,11]
bracket = {1:(0,5000), 2:(5000,9999), 3:(10000,14999), 4:(15000,19999), 5:(20000,24999), 6:(25000,34999), 7:(35000,49999),
           8:(50000,74999), 9:(75000,99999), 10:(100000,149999), 11:(150000,300000)}

#define function

def wage(df_wage,row):
    """returns randomly chosen income,
    weighted by income bracket probability in that taxi zone"""
    prob_list = []
    for i in df_wage.iloc[row, 0:]:
        prob_list.append(i)
    choice = np.random.choice(bucket, p=prob_list)
    return np.random.randint(bracket[choice][0], size=None)
    
#test     

wage(df, 4)   

ValueError: a and p must have same size